# CompLabNGS2024 Final Project

## Getting the data
Visit the [Gene Expression Omnibus (GEO) website](https://www.ncbi.nlm.nih.gov/geo/) and search for the accession number you were given. Review the information on the dataset and explore the publication that generated the data.

Write a short but inclusive description of the dataset you were given. Include name of the dataset, the publication, authors, number and condition of samples and any other relevant information to allow others to understand the dataset.

Dataset name: 	Bulk RNA-seq of mouse heart tissue for analyzing gene expression of myocardial infarction progression.
Publication: Downregulation of apoptotic repressor AVEN exacerbates cardiac injury after myocardial infarction
Authors: Peng Yu, Shuai Song, Xiaokai Zhang, Shujun Cui, Gang Wei1, Zihang Huang, Linqi Zeng, Ting Ni, Aijun Sun
accession number: GSE236374 (in GEO database), https://www.ncbi.nlm.nih.gov/Traces/study/?acc=PRJNA990722 (in SRA database)

The dataset comprises heart tissue samples from mice, to explore genetic features that influence MI progression.
Nine mice were included divided into three equal groups, three mice in each. Two groups underwent surgery to make models of myocardial infarction: first group samples were taken 7 days after surgery and second group samples were taken 28 days after surgery. Third group (control group), sham operation group - underwent surgery without causing myocardial infarction.
RNA-seq was performed on heart tissue samples from each mouse, generating 9 samples in total. These samples were then used to analyze gene expression in the context of myocardial infarction progression.

After completing the entire analysis, create a conda environment with all the necessary packages and their versions to run the analysis. In the cell below, write the command you used to create the environment and the command to activate it. The name of the environment should be `<first name>_<last name>`. You can use ```bash conda list``` to list all the packages and their versions in the environment.

Modify the example command below to include the packages you used in your analysis.

In [ ]:
#module load miniconda/miniconda3-4.7.12-environmentally
!conda create -n ngs_project \
    python=3.7 \
    pandas=1.0.3 \
    numpy=1.18.1 \
    matplotlib=3.1.3 \
    seaborn=0.13.2 \
    scikit-learn=0.22.1  \
    sra-tools=3.1.0  \
    fastqc=0.12.1  \
    trimmomatic=0.39  \
    fastuniq=1.1 \
    kallisto=0.50.1 \
    pydeseq2=0.4.7  \
    gseapy=1.1.3  \
!conda activate ngs_project

## Getting the raw fastq files
At the bottom of the GEO page, you will find a link to the `SRA Run Selector`. Click on the link. 

Explain what is the SRA and SRR in gerneal and how it is used to store sequencing data.


SRA (Sequence Read Archive) is a database maintained by NCBI that stores raw sequencing data. The format of the data is also called SRA. Each study/project in the SRA database is given an accession prefixed with SRP. For example, the reads of the study we chose to work on has the accession number SRP447124. The study is composed of sequencing experiments, prefixed with SRX, and each experiment might be conducted multiple times. Each run of the experiment is called a sequencing run, or SRR (Sequence Read Run). SRR is also the prefix of the unique identifier for a specific sequencing run. In our study, there is only 1 run per experiment, hence we ignore the SRX level and work directly with the SRR level. As explained above, we have 9 SRRs.

Use `prefetch + fasterq-dump` command from the `SRA Toolkit` to download the fastq files. Write the command you used to download the fastq files. [SRA Tools Conda installation](https://anaconda.org/bioconda/sra-tools)

In [7]:
import os
import subprocess
from multiprocessing import Pool

BASE_DIR = '/home/yair/ngs_project'
#BASE_DIR = '/urigo/Liat/ngs_course/'
srr_accessions = ['SRR25115639', 'SRR25115640', 'SRR25115641', 'SRR25115642', 'SRR25115643', 'SRR25115644', 'SRR25115645', 'SRR25115646', 'SRR25115647']

INPUT_FASTQ_FILES = os.path.join(BASE_DIR, 'input_fastq')
FASTQC_OUTPUTS_DIR = os.path.join(BASE_DIR, 'fastqc_outputs')
TRIMMOMATIC_OUTPUT_DIR = os.path.join(BASE_DIR, 'trimmomatic_outputs')
FASTUNIQ_INPUTS_FILES_DIR = os.path.join(BASE_DIR, 'fastuniq_inputs')
FASTUNIQ_OUTPUT_DIR = os.path.join(BASE_DIR, 'fastuniq_outputs')
CLEAN_READS_FASTQC_OUTPUTS_DIR = os.path.join(BASE_DIR, 'clean_reads_fastqc_outputs')

#generate folders
os.makedirs(INPUT_FASTQ_FILES, exist_ok=True)
os.makedirs(FASTQC_OUTPUTS_DIR, exist_ok=True)
os.makedirs(TRIMMOMATIC_OUTPUT_DIR, exist_ok=True)
os.makedirs(FASTUNIQ_INPUTS_FILES_DIR, exist_ok=True)
os.makedirs(FASTUNIQ_OUTPUT_DIR, exist_ok=True)
os.makedirs(CLEAN_READS_FASTQC_OUTPUTS_DIR, exist_ok=True)

In [ ]:
#download data from sra 
def download_fastq(srr_accession):
    os.chdir(INPUT_FASTQ_FILES)
    subprocess.run(f'prefetch {srr_accession}', shell=True)
    subprocess.run(f'fasterq-dump {srr_accession}', shell=True)
    os.rmdir(srr_accession) #delete srr folder

with Pool() as pool:
    pool.map(download_fastq, srr_accessions)

os.chdir(BASE_DIR)

## Quality Control
Below include all the commands, figures and output you used and generated to perform quality control on the fastq files. You can use `fastqc`, `trimmomatic`, `multiqc` and any other tool you find necessary to give a comprehensive QC and filtering of the data.

Write a brief description of the quality control process and explain the figures and tables you generated.

In [ ]:
# First we run FastQC on 3 of the input fastq files (1 for each of the three groups).
srr_acessions_representatives = ['SRR25115639', 'SRR25115642', 'SRR25115645']

def run_fastqc(srr_accession):
    srr_1_path = os.path.join(INPUT_FASTQ_FILES, f'{srr_accession}_1.fastq')
    srr_2_path = os.path.join(INPUT_FASTQ_FILES, f'{srr_accession}_2.fastq')
    subprocess.run(f'fastqc {srr_1_path} {srr_2_path} -o {FASTQC_OUTPUTS_DIR}', shell=True)

with Pool() as pool:
    pool.map(run_fastqc, srr_acessions_representatives)

We examined the fastqc reports and noticed 2 major issues:
* Per base sequence content tab - the first 10 bases aren't distributed according to the rest of the read sequence. That is unexpected and therefore we will use trimmomatic HEADCROP module to cut the first 10 bases of each read. We will also follow conventional trimming practices and trim reads shorter than 90 bases.
* Sequence Duplication Levels tab - this tab indicated that there are duplicated reads. Therefore we will use fastuniq to remove duplicated reads.

![fastqc_per_base_sequence_content.jpg](fastqc_raw_reads/SRR25115639_1_per_base_sequence_content.png)
![fastqc_duplication_levels.jpg](fastqc_raw_reads/SRR25115639_1_sequence_duplication_levels.png)

In [ ]:
# Now we will run trimmomatic and fastuniq

def run_trimmomatic(srr_accession):
    srr_1_path = os.path.join(INPUT_FASTQ_FILES, f'{srr_accession}_1.fastq')
    srr_2_path = os.path.join(INPUT_FASTQ_FILES, f'{srr_accession}_2.fastq')
    trimmomatic_output_path = os.path.join(TRIMMOMATIC_OUTPUT_DIR, f'{srr_accession}.fastq')
    subprocess.run(f'trimmomatic PE {srr_1_path} {srr_2_path} -baseout {trimmomatic_output_path} HEADCROP:10 MINLEN:90', shell=True)

with Pool() as pool:
    pool.map(run_trimmomatic, srr_accessions)

In [ ]:
def run_fastuniq(srr_accession):
    #generate file with the paths of the trimmomatic outputs
    fastuniq_input_path = os.path.join(FASTUNIQ_INPUTS_FILES_DIR, f'{srr_accession}.txt')
    with open(fastuniq_input_path, 'w') as fp:
        fp.write(os.path.join(TRIMMOMATIC_OUTPUT_DIR, f'{srr_accession}_1P.fastq') + '\n')
        fp.write(os.path.join(TRIMMOMATIC_OUTPUT_DIR, f'{srr_accession}_2P.fastq'))
    
    fastuniq_1_output = os.path.join(FASTUNIQ_OUTPUT_DIR, f'{srr_accession}_1.fastq')
    fastuniq_2_output = os.path.join(FASTUNIQ_OUTPUT_DIR, f'{srr_accession}_2.fastq')
    subprocess.run(f'fastuniq -i {fastuniq_input_path} -o {fastuniq_1_output} -p {fastuniq_2_output}', shell=True)

with Pool() as pool:
    pool.map(run_fastuniq, srr_accessions)

In [ ]:
# Finally we run Fastqc again on the representative fastq files
def run_fastqc_on_clean_reads(srr_accession):
    srr_1_path = os.path.join(FASTUNIQ_OUTPUT_DIR, f'{srr_accession}_1.fastq')
    srr_2_path = os.path.join(FASTUNIQ_OUTPUT_DIR, f'{srr_accession}_2.fastq')
    subprocess.run(f'fastqc {srr_1_path} {srr_2_path} -o {CLEAN_READS_FASTQC_OUTPUTS_DIR}', shell=True)

with Pool() as pool:
    pool.map(run_fastqc_on_clean_reads, srr_acessions_representatives)

Now we will examine the fastqc reports of the clean reads. We can see that the issues we identified in the raw reads are no longer present in the clean reads.
Moreover, in the forward read of the first sample (SRR2511639) the number of reads went down from 23194384 to 16893355 (27% reduction).

![fastqc_per_base_sequence_content.jpg](fastqc_clean_reads/SRR25115639_1_per_base_sequence_content.png)
![fastqc_duplication_levels.jpg](fastqc_clean_reads/SRR25115639_1_sequence_duplication_levels.png)

## Alignment and DGE

Similar to what we did in class and in the assignemnts, you will now run through the steps of mapping and conducting DGE. Include below an elaborate report of your analysis, include study desgin, example count and normalized count matrices, draw summary statistics and every other information you think is relevent to report your findings (if any)

Running `STAR` on a laptop or PC is almost impossible we will use one of the more recent pseudo alighners, ['kallisto`](https://pachterlab.github.io/kallisto/manual), which will allow us to quantify transcript level information in a more efficent manner.

1. Download the index files from kallisto's GitHub
```bash
wget https://github.com/pachterlab/kallisto-transcriptome-indices/releases/download/v1/mouse_index_standard.tar.xz
tar -xf mouse_index_standard.tar.xz
```
2. Install and run `kallisto`
```bash
conda install -c bioconda -c conda-forge kallisto==0.50.1
kallisto quant -i index.idx -o output -t 16 sample.fastq 
```

In the `output` directory you should have `abundance.tsv`. View the file and make sure you understand it. Produce quantification to all your samples.
We now need to sum up transcript level counts to gene level counts before proceeding to `pyDESEQ`

### Differential Gene Expression Analysis
Now we're going to use kallisto to extract the gene-level abundance from the reads. We're going to construct 2 tables: one with the TPM values ("all_samples_gene_abundance_counts.csv") and one with the counts values ("all_samples_gene_abundance_counts.csv"). Then we're going to use the counts table to filter out genes with low read counts, and then use the TPM table to run pyDESeq2 to identify differentially expressed genes between the groups.

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats

MOUSE_INDEX_DIR = os.path.join(BASE_DIR, 'mouse_index')
KALLISTO_OUTPUT_DIR = os.path.join(BASE_DIR, 'kallisto_outputs')
GENE_ABUNDANCES_DIR = os.path.join(BASE_DIR, 'gene_abundances')
PYDESEQ_DIR = os.path.join(BASE_DIR, 'pydeseq')

os.makedirs(MOUSE_INDEX_DIR, exist_ok=True)
os.makedirs(KALLISTO_OUTPUT_DIR, exist_ok=True)
os.makedirs(GENE_ABUNDANCES_DIR, exist_ok=True)
os.makedirs(PYDESEQ_DIR, exist_ok=True)

In [ ]:
# Download the index files from kallisto's GitHub
os.chdir(MOUSE_INDEX_DIR)

subprocess.run(f'wget https://github.com/pachterlab/kallisto-transcriptome-indices/releases/download/v1/mouse_index_standard.tar.xz', shell=True)
subprocess.run(f'tar -xf mouse_index_standard.tar.xz', shell=True)

In [ ]:
# Run kallisto quant on all samples
def run_kallisto(srr_accession):
    srr_1_path = os.path.join(FASTUNIQ_OUTPUT_DIR, f'{srr_accession}_1.fastq')
    srr_2_path = os.path.join(FASTUNIQ_OUTPUT_DIR, f'{srr_accession}_2.fastq')

    kallisto_output_path = os.path.join(KALLISTO_OUTPUT_DIR, srr_accession)
    os.makedirs(kallisto_output_path, exist_ok=True)

    subprocess.run(f'kallisto quant -i {os.path.join(MOUSE_INDEX_DIR, "index.idx")} -o {kallisto_output_path} -t 16 {srr_1_path} {srr_2_path}', shell=True)

with Pool(processes=3) as pool:
    pool.map(run_kallisto, srr_accessions)

In [ ]:
# Load transcript to gene mapping file
t2g_file = os.path.join(MOUSE_INDEX_DIR, 't2g.txt')
t2g_df = pd.read_csv(t2g_file, sep='\t', usecols=[0, 2], header=None, names=['transcript_id', 'gene_id'])

def calculate_gene_abundance(srr_accession):
    abundance_file = os.path.join(KALLISTO_OUTPUT_DIR, srr_accession, 'abundance.tsv')
    abundance_df = pd.read_csv(abundance_file, sep='\t')

    # Merge transcript abundance with gene mapping
    merged_df = pd.merge(abundance_df, t2g_df, left_on='target_id', right_on='transcript_id', how='inner')

    # Sum abundances at the gene level
    gene_abundance_tpm_df = merged_df.groupby('gene_id')['tpm'].sum().reset_index()
    gene_abundance_counts_df = merged_df.groupby('gene_id')['est_counts'].sum().reset_index()

    # Rename tpm column to the srr accession
    gene_abundance_tpm_df.rename(columns={'tpm': srr_accession}, inplace=True)
    gene_abundance_counts_df.rename(columns={'est_counts': srr_accession}, inplace=True)

    # Save gene-level abundance to a new file
    gene_abundance_tpm_df.to_csv(os.path.join(GENE_ABUNDANCES_DIR, f'{srr_accession}_gene_abundance_tpm.csv'), index=False)
    gene_abundance_counts_df.to_csv(os.path.join(GENE_ABUNDANCES_DIR, f'{srr_accession}_gene_abundance_counts.csv'), index=False)

with Pool() as pool:
    pool.map(calculate_gene_abundance, srr_accessions)


Do you understand the code? Include `print` statments and rerun until you understnad what's going on fully

Do this operation to each of your sample's abundance file. Using python or a different tool merge all samples to a single file.

You should now be able to take the resulting `all_samples_gene_abundance.csv` and use it in `pyDeSeq2` as we did in class

In [ ]:
# Merge all gene abundance files to a single file

gene_abundance_tpm_files = [os.path.join(GENE_ABUNDANCES_DIR, f'{srr_accession}_gene_abundance_tpm.csv') for srr_accession in srr_accessions]
gene_abundance_counts_files = [os.path.join(GENE_ABUNDANCES_DIR, f'{srr_accession}_gene_abundance_counts.csv') for srr_accession in srr_accessions]

gene_abundance_tpm_dfs = [pd.read_csv(file_path, index_col='gene_id') for file_path in gene_abundance_tpm_files]
gene_abundance_counts_dfs = [pd.read_csv(file_path, index_col='gene_id') for file_path in gene_abundance_counts_files]

all_samples_gene_abundance_tpm_df = pd.concat(gene_abundance_tpm_dfs, axis=1)
all_samples_gene_abundance_counts_df = pd.concat(gene_abundance_counts_dfs, axis=1)

all_samples_gene_abundance_tpm_df.to_csv(os.path.join(GENE_ABUNDANCES_DIR, 'all_samples_gene_abundance_tpm.csv'))
all_samples_gene_abundance_counts_df.to_csv(os.path.join(GENE_ABUNDANCES_DIR, 'all_samples_gene_abundance_counts.csv'))

Let's show the first few rows of the TPM matrix and the counts matrix.

In [5]:
all_samples_gene_abundance_tpm_df = pd.read_csv(r"gene_abundances_tables\all_samples_gene_abundance_tpm.csv", index_col='gene_id')
all_samples_gene_abundance_counts_df = pd.read_csv(r"gene_abundances_tables\all_samples_gene_abundance_counts.csv", index_col='gene_id')
print(all_samples_gene_abundance_tpm_df.head())
print(all_samples_gene_abundance_counts_df.head())

               SRR25115639  SRR25115640  SRR25115641  SRR25115642  \
gene_id                                                             
0610005C13Rik     0.091880     0.391548     0.259823     0.406482   
0610006L08Rik     0.000000     0.000000     0.000000     0.000000   
0610009B22Rik    26.706800    25.845500    32.464000    32.212300   
0610009E02Rik     7.223749     8.516500     5.150168    15.512649   
0610009L18Rik     9.716101    12.848902     9.324906     9.261853   

               SRR25115643  SRR25115644  SRR25115645  SRR25115646  SRR25115647  
gene_id                                                                         
0610005C13Rik     0.460475     0.462845     0.257156     0.407720     0.668718  
0610006L08Rik     0.000000     0.000000     0.000000     0.000000     0.000000  
0610009B22Rik    27.473100    27.418100    34.786500    34.490700    34.086004  
0610009E02Rik    13.930147    18.631090     2.184616     2.217021     3.374383  
0610009L18Rik     8.279300    

In [ ]:
# Now we use the effective counts matrix to identify genes with low read counts, and filter them out from the tpm matrix. This is done to reduce bias, since genes with a small read count might have a high misleading log2-fold-change. We chose 100 as the minimum read count threshold, after examining multiple options and choosing one the reduces the number of outliers.
MIN_READ_COUNT = 100

all_samples_gene_abundance_counts_df = pd.read_csv(os.path.join(GENE_ABUNDANCES_DIR, 'all_samples_gene_abundance_counts.csv'), index_col='gene_id')
all_samples_gene_abundance_tpm_df = pd.read_csv(os.path.join(GENE_ABUNDANCES_DIR, 'all_samples_gene_abundance_tpm.csv'), index_col='gene_id')

all_samples_gene_abundance_counts_filtered_df = all_samples_gene_abundance_counts_df[(all_samples_gene_abundance_counts_df >= MIN_READ_COUNT).any(axis=1)]
significant_genes = all_samples_gene_abundance_counts_filtered_df.index
print("Number of genes before filtering:", all_samples_gene_abundance_tpm_df.shape[0])
print(f"Number of genes after filtering genes with at least {MIN_READ_COUNT} reads in at least one sample:", len(significant_genes))

all_samples_gene_abundance_df = all_samples_gene_abundance_tpm_df.loc[significant_genes]
all_samples_gene_abundance_df.to_csv(os.path.join(GENE_ABUNDANCES_DIR, 'all_samples_gene_abundance.csv'))

Number of genes before filtering: 32621
Number of genes after filtering genes with at least 100 reads in at least one sample: 12178

In [ ]:
# Clustermap the count matrix

all_samples_gene_abundance_df = pd.read_csv(os.path.join(GENE_ABUNDANCES_DIR, 'all_samples_gene_abundance.csv'), index_col='gene_id')
cluster_plot = sns.clustermap(all_samples_gene_abundance_df.corr(), cmap='coolwarm')
cluster_plot.savefig(os.path.join(GENE_ABUNDANCES_DIR, 'samples_clustermap.png'))
plt.clf()


![samples_clustermap.png](samples_clustermap.png)

We clustered the samples based on the correlation of their gene expression levels. First, as expected, we see that the samples of each group are clustered together. Besides that, we can see that the samples of the 28 days group are more similar to the control (sham) group than to the 7 days group. This can be explained by the fact that the 28 days group samples were taken after a longer period of time, and thus the mice had more time to recover from the surgery and the myocardial infarction (and this led to gene expression levels that are more similar to the control group).

Next, we'll use pyDESeq2 to identify differentially expressed genes (DGE) between the groups.
Our goal is to identify the genes that are differentially expressed between before and after the treatment. Since we can see that with regard to the sham group, the highest effect of the treatment is in the 7-days group, we'll do our DGE analysis on the 7-days samples vs. the sham samples.

In [ ]:
# Run pyDESeq2

samples = ['SRR25115645', 'SRR25115646', 'SRR25115647', 'SRR25115642', 'SRR25115643', 'SRR25115644']
study_design_df = pd.DataFrame({
    'sample': samples,
    'group': ['sham', 'sham', 'sham', '7 days', '7 days', '7 days']
})
study_design_df.set_index('sample', inplace=True)
study_design_df.to_csv(os.path.join(PYDESEQ_DIR, 'study_design.csv'), index=False)

# We multiply the gene expression levels by 1e6 to get integer counts, which is required by pyDESeq2.
all_samples_gene_abundance_df = pd.read_csv(os.path.join(GENE_ABUNDANCES_DIR, 'all_samples_gene_abundance.csv'), index_col='gene_id')
all_samples_gene_abundance_df = all_samples_gene_abundance_df[samples]
deseq_counts_input = (all_samples_gene_abundance_df * 1e6).astype(int).T
deseq_counts_input.to_csv(os.path.join(PYDESEQ_DIR, 'deseq_counts_input.csv'))

inference = DefaultInference(n_cpus=8)
dds = DeseqDataSet(
    counts=deseq_counts_input,
    metadata=study_design_df,
    design_factors=['group'],
    refit_cooks=True,
    inference=inference
)

dds.deseq2()

stats_results = DeseqStats(dds, inference=inference)
print(stats_results.summary())

stats_results.results_df.to_csv(os.path.join(PYDESEQ_DIR, "stats_results.csv"))

After running pyDESeq2, we can see the results of the DGE analysis in the stats_results.csv file. This file contains the log2 fold change, p-value, and adjusted p-value for each gene. We can use this information to identify the genes that are differentially expressed between the 7-days group and the sham group.

In [ ]:
# First, let's create a volcano plot to visualize the results
stats_df = pd.read_csv(os.path.join(PYDESEQ_DIR, "stats_results.csv"))
stats_df['-log10pvalue'] = -np.log10(stats_df['padj'])
volcano_plot = sns.scatterplot(data=stats_df, x='log2FoldChange', y='-log10pvalue', s=1)

# Add significance threshold lines, and limit the x-axis to be symmetrical around 0.
significant_pvalue = 0.01
volcano_plot.axhline(-np.log10(significant_pvalue), color='gray', linestyle='--', linewidth=1)
volcano_plot.axvline(-1, color='gray', linestyle='--', linewidth=1)
volcano_plot.axvline(1, color='gray', linestyle='--', linewidth=1)
x_limit = min(abs(min(stats_df['log2FoldChange'])), abs(max(stats_df['log2FoldChange'])))
volcano_plot.set_xlim(-x_limit, x_limit)

plt.tight_layout()
plt.savefig(os.path.join(PYDESEQ_DIR, "volcano_plot.png"), dpi=300, bbox_inches='tight')

# Now let's identify how many genes are differentially expressed
genes_expressed_higher_in_treatment = stats_df.loc[(stats_df['log2FoldChange'] > 1) & (stats_df['padj'] < significant_pvalue)]
genes_expressed_higher_in_sham = stats_df.loc[(stats_df['log2FoldChange'] < -1) & (stats_df['padj'] < significant_pvalue)]
treatment_higher = len(genes_expressed_higher_in_treatment)
sham_higher = len(genes_expressed_higher_in_sham)
differentially_expressed_genes = treatment_higher + sham_higher

print(f"There are {differentially_expressed_genes} DE genes, of them {treatment_higher} are expressed higher in the treatment group and {sham_higher} are expressed higher in the sham (control) group.")

There are 4647 DE genes, of them 2105 are expressed higher in the treatment group and 2542 are expressed higher in the sham (control) group.

The volcano plot shows the log2 fold change vs. the -log10 p-value for each gene. The dashed lines represent the significance threshold for the p-value and the log2 fold change. We limited the x-axis to be symmetrical around 0 to better visualize the results. The plot shows that there are many genes with a high log2 fold change and a low p-value, indicating that they are differentially expressed between the 7-days group and the sham group.
![volcano_plot.png](volcano_plot.png)

## **BONUS** Gene Set Enrichment Analysis

Up until this point we covered material learned in class. If you want to boost your scores attempt at learning and running GSEA analysis on your DGE results.

Explain what GSEA is and what insights can we draw from it.

Execute a GSEA analysis using [pyGSEA](https://gseapy.readthedocs.io/en/latest/introduction.html) and summarize your results below.

GSEA (=Gene Set Enrichment Analysis) is a computational method that determines whether an a priori defined set of genes shows statistically significant, concordant differences between two biological states. It is used to interpret gene expression data by analyzing gene sets rather than individual genes. GSEA calculates an enrichment score for each gene set, which is a measure of the overrepresentation of the gene set at the top or bottom of the ranked list of genes. The significance of the enrichment score is calculated by permuting the gene labels many times to create a null distribution of enrichment scores. The enrichment score is then normalized to account for the size of the gene set.

In [ ]:
import os
import pandas as pd

from gseapy.plot import gseaplot
import gseapy as gp

GSEA_OUTPUT_DIR = os.path.join(BASE_DIR, 'gsea')
os.makedirs(GSEA_OUTPUT_DIR, exist_ok=True)

dge_results = pd.read_csv(os.path.join(PYDESEQ_DIR, "stats_results.csv")).dropna()
dge_results['Rank'] = -np.log10(dge_results.padj) * dge_results.log2FoldChange

# Replace inf values in Rank column with the maximum value in the column + 1,
# and -inf values with the minimum value in the column -1 (to avoid errors in the GSEA analysis)
max_rank = dge_results['Rank'].loc[dge_results['Rank'] != np.inf].max()
min_rank = dge_results['Rank'].loc[dge_results['Rank'] != -np.inf].min()
dge_results['Rank'] = dge_results['Rank'].replace(np.inf, max_rank + 1)
dge_results['Rank'] = dge_results['Rank'].replace(-np.inf, min_rank - 1)

dge_results = dge_results.sort_values('Rank', ascending = False).reset_index(drop = True)
dge_results['gene_id'] = dge_results['gene_id'].str.upper()
dge_results.to_csv(os.path.join(GSEA_OUTPUT_DIR, 'dge_results_ranked.csv'))

genes_ranking = dge_results[['gene_id', 'Rank']]
pre_res = gp.prerank(rnk = genes_ranking, gene_sets='GO_Biological_Process_2021', seed=6, permutation_num = 100)
pre_res.res2d.to_csv(os.path.join(GSEA_OUTPUT_DIR, 'go_processes_enrichment_results.csv'))

significant_terms = pre_res.res2d.loc[pre_res.res2d['FDR q-val'] < 0.05]['Term'].to_list()
print(f"The significant terms are: {significant_terms}")

most_up_regulated_significant_term = pre_res.res2d.loc[pre_res.res2d['NES'] > 0].iloc[0]['Term']
most_down_regulated_significant_term = pre_res.res2d.loc[pre_res.res2d['NES'] < 0].iloc[0]['Term']

print(f"Plotting ES plot for the most up regulated significant term: {most_up_regulated_significant_term}")
gseaplot(rank_metric=pre_res.ranking, term=most_up_regulated_significant_term, ofname=os.path.join(GSEA_OUTPUT_DIR, 'upregulated_term_plot.pdf'), **pre_res.results[most_up_regulated_significant_term])

print(f"Plotting ES plot for the most down regulated significant term: {most_down_regulated_significant_term}")
gseaplot(rank_metric=pre_res.ranking, term=most_down_regulated_significant_term, ofname=os.path.join(GSEA_OUTPUT_DIR, 'downregulated_term_plot.pdf'), **pre_res.results[most_down_regulated_significant_term])


The significant terms are: ['extracellular matrix organization (GO:0030198)', 'extracellular structure organization (GO:0043062)', 'external encapsulating structure organization (GO:0045229)', 'mitochondrial translation (GO:0032543)', 'neutrophil degranulation (GO:0043312)', 'neutrophil activation involved in immune response (GO:0002283)', 'neutrophil mediated immunity (GO:0002446)', 'supramolecular fiber organization (GO:0097435)', 'aerobic electron transport chain (GO:0019646)', 'glucose metabolic process (GO:0006006)', 'mitochondrial ATP synthesis coupled electron transport (GO:0042775)', 'collagen fibril organization (GO:0030199)', 'mitochondrial translational elongation (GO:0070125)', 'mitochondrial translational termination (GO:0070126)', 'dicarboxylic acid metabolic process (GO:0043648)', 'mitochondrion organization (GO:0007005)', 'translational elongation (GO:0006414)', 'mitochondrial respiratory chain complex assembly (GO:0033108)', 'cellular respiration (GO:0045333)', 'negative regulation of canonical Wnt signaling pathway (GO:0090090)', 'regulation of immune response (GO:0050776)', 'cellular response to cytokine stimulus (GO:0071345)', 'translational termination (GO:0006415)', 'negative regulation of Wnt signaling pathway (GO:0030178)', 'negative regulation of angiogenesis (GO:0016525)', 'fatty acid beta-oxidation (GO:0006635)', 'cytokine-mediated signaling pathway (GO:0019221)', 'regulation of cellular ketone metabolic process (GO:0010565)', 'lipid droplet organization (GO:0034389)', 'positive regulation of immune response (GO:0050778)', 'fatty acid catabolic process (GO:0009062)', 'cellular response to transforming growth factor beta stimulus (GO:0071560)', 'mitochondrial electron transport, NADH to ubiquinone (GO:0006120)', 'negative regulation of blood vessel morphogenesis (GO:2000181)', 'positive regulation of cytokine production (GO:0001819)', 'mitochondrial respiratory chain complex I assembly (GO:0032981)', 'NADH dehydrogenase complex assembly (GO:0010257)', 'inflammatory response (GO:0006954)', 'positive regulation of response to external stimulus (GO:0032103)', 'fatty acid metabolic process (GO:0006631)', 'positive regulation of cell migration (GO:0030335)', 'regulation of angiogenesis (GO:0045765)', 'fatty acid oxidation (GO:0019395)', 'regulation of glucose metabolic process (GO:0010906)', 'regulation of mRNA processing (GO:0050684)', 'transforming growth factor beta receptor signaling pathway (GO:0007179)', 'peptidyl-lysine modification (GO:0018205)', 'regulation of Wnt signaling pathway (GO:0030111)', 'canonical Wnt signaling pathway (GO:0060070)', 'transmembrane receptor protein serine/threonine kinase signaling pathway (GO:0007178)', 'negative regulation of autophagy (GO:0010507)', "mRNA 3'-end processing (GO:0031124)", 'cellular response to molecule of bacterial origin (GO:0071219)', 'Wnt signaling pathway (GO:0016055)', 'regulation of inflammatory response (GO:0050727)', 'response to interferon-gamma (GO:0034341)', 'regulation of epithelial to mesenchymal transition (GO:0010717)', 'toll-like receptor signaling pathway (GO:0002224)', 'innate immune response (GO:0045087)', 'response to cytokine (GO:0034097)', 'positive regulation of signal transduction (GO:0009967)', 'innate immune response activating cell surface receptor signaling pathway (GO:0002220)', 'cellular response to lectin (GO:1990858)', 'stimulatory C-type lectin receptor signaling pathway (GO:0002223)', 'extracellular matrix disassembly (GO:0022617)', 'cellular component disassembly (GO:0022411)', 'negative regulation of multicellular organismal process (GO:0051241)', 'cellular response to lipopolysaccharide (GO:0071222)', 'proteolysis (GO:0006508)', 'regulation of cytokine production (GO:0001817)', 'positive regulation of defense response (GO:0031349)', 'non-canonical Wnt signaling pathway (GO:0035567)', 'glycosaminoglycan catabolic process (GO:0006027)', 'positive regulation of cell population proliferation (GO:0008284)', 'positive regulation of establishment of protein localization (GO:1904951)', 'glycolipid metabolic process (GO:0006664)', 'regulation of cell adhesion mediated by integrin (GO:0033628)', 'positive regulation of receptor signaling pathway via JAK-STAT (GO:0046427)', 'regulation of peptidyl-tyrosine phosphorylation (GO:0050730)', 'regulation of Rho protein signal transduction (GO:0035023)', 'regulation of cell migration (GO:0030334)', 'regulation of innate immune response (GO:0045088)', 'mesenchymal cell differentiation (GO:0048762)', 'Fc-gamma receptor signaling pathway (GO:0038094)', 'RNA splicing, via transesterification reactions with bulged adenosine as nucleophile (GO:0000377)', 'positive regulation of cell motility (GO:2000147)', 'response to tumor necrosis factor (GO:0034612)', 'mRNA polyadenylation (GO:0006378)', 'RNA polyadenylation (GO:0043631)', 'regulation of defense response (GO:0031347)']

Plotting ES plot for the most up regulated significant term: mitochondrial translation (GO:0032543)
Plotting ES plot for the most down regulated significant term: extracellular matrix organization (GO:0030198)

We can see that the most upregulated term is "mitochondrial translation" and the most downregulated term is "extracellular matrix organization". The ES plot of "mitochondrial translation" shows that the genes in this term are enriched at the top of the ranked list of genes, while the ES plot of "extracellular matrix organization" shows that the genes in this term are enriched at the bottom of the ranked list of genes. This indicates that the genes in the "mitochondrial translation" term are upregulated in the 7-days group compared to the sham group, while the genes in the "extracellular matrix organization" term are downregulated.

![upregulated_term_plot.png](gsea/upregulated_term_plot.png)
![downregulated_term_plot.png](gsea/downregulated_term_plot.png)

## Notes

This excerise is meant to 'through' in the water and might prove challanging. Attempt to work in groups to overcome challanges. 

If you get stuck try to post an issue, but give as much background and explanation of your problem as possible so I could help

### Good luck!